<a href="https://colab.research.google.com/github/BowenZheng2003/Capstone-2T6/blob/main/Audio_Stream/Segmenting_Audio_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install opensmile xgboost librosa soundfile pandas scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.0/996.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.9/324.9 kB 32.0 MB/s eta 0:00:00


In [4]:
from pydub import AudioSegment
import os
import opensmile
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import joblib

In [7]:
# Location of all of the interview audio files
audio_folder = "/content/drive/MyDrive/Audio Files"
save_folder = "/content/drive/MyDrive/Segmented Interview Information"

# Initialize OpenSMILE
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals
)

In [8]:
# Set durations
segment_duration_ms = 5000
overlap_ms = 2500 # 50% overlap
step_size = segment_duration_ms - overlap_ms

# Iterate through each .wav file in the folder
for filename in os.listdir(audio_folder):
    if filename.endswith(".wav"):
        file_path = os.path.join(audio_folder, filename)
        audio = AudioSegment.from_wav(file_path)
        file_id = os.path.splitext(filename)[0]

        results = []

        for i in range(0, len(audio) - segment_duration_ms + 1, step_size):
            segment = audio[i:i + segment_duration_ms]
            segment_path = f"/tmp/tmp_{file_id}_{i}.wav"
            segment.export(segment_path, format="wav")

            # Extract all eGeMAPSv02 features
            features_df = smile.process_file(segment_path).reset_index(drop=True)
            features_dict = features_df.round(3).to_dict("records")[0]

            # Timestamp for segment
            start_sec = i // 1000
            end_sec = (i + segment_duration_ms) // 1000
            timestamp = f"{start_sec//60:02d}:{start_sec%60:02d} - {end_sec//60:02d}:{end_sec%60:02d}"

            # Add metadata
            features_dict["file_id"] = file_id
            features_dict["timestamp"] = timestamp

            results.append(features_dict)
            os.remove(segment_path)

        # Save full eGeMAPSv02 feature set per file
        df = pd.DataFrame(results)
        output_path = os.path.join(save_folder, f"{file_id}_all_features.csv")
        df.to_csv(output_path, index=False)

        # Confirm output
        print(f"✅ Processed and saved {len(df)} segments for '{file_id}'")
        print(f"📄 Saved to: {output_path}")
        print("📊 Sample features:")
        display(df.head())
        print("="*60)

✅ Processed and saved 162 segments for 'PP71'
📄 Saved to: /content/drive/MyDrive/Segmented Interview Information/PP71_all_features.csv
📊 Sample features:


,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,file_id,timestamp
0,17.013000,0.374,13.501,14.577000,17.490999,3.990000,153.850006,191.093994,95.846001,67.524002,...,0.082,2.004,5.274,0.088,0.081,0.084,0.072,-47.542999,PP71,00:00 - 00:05
1,17.438000,0.451,13.452,14.352000,16.555000,3.103000,190.651001,237.205994,84.225998,78.420998,...,0.084,1.804,4.057,0.152,0.139,0.082,0.073,-48.169998,PP71,00:02 - 00:07
2,25.077999,0.482,13.807,17.363001,39.537998,25.731001,386.937012,812.172974,88.176003,64.146004,...,0.070,3.407,3.043,0.246,0.167,0.076,0.046,-45.811001,PP71,00:05 - 00:10
3,30.358999,0.413,15.443,37.312000,40.362000,24.919001,828.981018,1107.348999,141.274994,94.182999,...,0.056,4.409,3.870,0.153,0.105,0.106,0.101,-46.250999,PP71,00:07 - 00:12
4,25.131001,0.494,13.460,21.562000,35.854000,22.395000,486.626007,614.817993,97.649002,56.257000,...,0.068,3.808,4.665,0.100,0.076,0.118,0.130,-47.098999,PP71,00:10 - 00:15


✅ Processed and saved 126 segments for 'PP69'
📄 Saved to: /content/drive/MyDrive/Segmented Interview Information/PP69_all_features.csv
📊 Sample features:


,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,file_id,timestamp
0,23.392000,0.255,19.878,21.964001,31.777000,11.900,132.889008,106.522003,62.896000,70.278999,...,0.082,2.405,2.632,0.272,0.314,0.095,0.062,-40.903000,PP69,00:00 - 00:05
1,20.753000,0.239,14.920,21.315001,22.919001,7.999,106.099998,119.292000,34.896000,23.485001,...,0.065,1.403,2.632,0.268,0.345,0.107,0.086,-39.865002,PP69,00:02 - 00:07
2,20.395000,0.231,15.078,21.101999,22.896000,7.818,150.955994,104.141998,81.834999,73.250999,...,0.074,2.405,2.840,0.267,0.296,0.075,0.067,-39.075001,PP69,00:05 - 00:10
3,19.982000,0.235,16.528,19.773001,21.207001,4.679,178.882996,116.365997,102.032997,94.807999,...,0.093,4.609,1.826,0.478,0.363,0.064,0.027,-40.418999,PP69,00:07 - 00:12
4,18.070999,0.199,15.280,18.503000,19.791000,4.511,181.947006,170.153000,69.072998,88.213997,...,0.110,4.208,2.434,0.357,0.337,0.044,0.028,-41.500999,PP69,00:10 - 00:15


✅ Processed and saved 157 segments for 'PP70'
📄 Saved to: /content/drive/MyDrive/Segmented Interview Information/PP70_all_features.csv
📊 Sample features:


...

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,file_id,timestamp
0,21.978001,0.395,13.922,17.076000,33.409000,19.487,185.181000,116.469002,111.344002,83.490997,...,0.081,2.004,4.868,0.087,0.081,0.122,0.098,-39.535999,P4,00:00 - 00:05
1,22.768000,0.313,15.043,23.337000,28.431999,13.389,213.042007,146.475006,113.194000,89.944000,...,0.085,2.605,4.665,0.126,0.114,0.092,0.090,-39.933998,P4,00:02 - 00:07
2,23.212999,0.254,15.987,24.764000,26.806999,10.819,391.815002,428.226013,104.857002,82.314003,...,0.122,3.206,3.055,0.247,0.203,0.072,0.047,-39.347000,P4,00:05 - 00:10
3,21.768999,0.282,14.465,23.604000,25.396999,10.933,341.675995,403.256012,106.169998,81.654999,...,0.088,4.008,3.043,0.249,0.273,0.065,0.044,-42.245998,P4,00:07 - 00:12
4,21.507999,0.282,14.534,23.589001,24.414000,9.880,311.813995,593.752991,92.484001,81.248001,...,0.060,2.004,3.455,0.215,0.238,0.061,0.042,-42.445000,P4,00:10 - 00:15


✅ Processed and saved 77 segments for 'P1'
📄 Saved to: /content/drive/MyDrive/Segmented Interview Information/P1_all_features.csv
📊 Sample features:


,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,file_id,timestamp
0,21.209999,0.352,13.557,17.680000,26.819000,13.261000,88.448997,60.480999,72.803001,33.421001,...,0.057,1.804,2.851,0.246,0.229,0.079,0.062,-38.627998,P1,00:00 - 00:05
1,20.958000,0.381,13.912,16.980000,28.167999,14.255000,161.356995,211.897003,71.642998,67.874001,...,0.077,1.202,4.049,0.164,0.163,0.067,0.055,-38.883999,P1,00:02 - 00:07
2,23.924000,0.338,14.610,26.158001,31.236000,16.627001,280.054993,228.272003,87.225998,73.944000,...,0.079,1.403,4.260,0.152,0.170,0.075,0.057,-31.730000,P1,00:05 - 00:10
3,23.309999,0.281,14.206,26.650000,28.035000,13.829000,215.278000,168.531998,71.703003,48.590000,...,0.076,2.405,2.834,0.258,0.272,0.086,0.071,-31.072001,P1,00:07 - 00:12
4,25.292999,0.255,18.414,27.132000,28.229000,9.815000,486.391998,694.882019,86.778999,63.993000,...,0.132,3.006,2.434,0.286,0.342,0.139,0.083,-32.888000,P1,00:10 - 00:15


✅ Processed and saved 188 segments for 'P3'
📄 Saved to: /content/drive/MyDrive/Segmented Interview Information/P3_all_features.csv
📊 Sample features:


,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,file_id,timestamp
0,23.365000,0.342,14.432000,24.330000,31.493000,17.061001,135.858994,78.068001,107.954002,95.732002,...,0.203,1.804,3.846,0.163,0.169,0.081,0.078,-30.238001,P3,00:00 - 00:05
1,25.146999,0.329,15.821000,26.229000,32.931000,17.110001,185.983002,288.828003,134.100998,105.565002,...,0.255,2.004,4.057,0.160,0.130,0.070,0.064,-29.857000,P3,00:02 - 00:07
2,24.757999,0.295,17.955999,24.403000,30.475000,12.519000,416.994995,720.036011,113.911003,90.596001,...,0.254,2.806,3.462,0.174,0.129,0.095,0.066,-29.410000,P3,00:05 - 00:10
3,23.145000,0.312,16.672001,21.471001,28.302999,11.631000,362.410004,685.000977,132.067993,89.011002,...,0.383,3.407,3.854,0.152,0.154,0.088,0.089,-29.334999,P3,00:07 - 00:12
4,21.639999,0.324,15.442000,20.993999,26.108999,10.666000,244.419006,224.399994,176.214005,139.740005,...,0.316,3.407,5.274,0.097,0.106,0.085,0.081,-32.694000,P3,00:10 - 00:15


In [28]:
df = pd.read_csv("/content/drive/MyDrive/Segmented Interview Information/P1_all_features.csv")

# Example: only use a subset of features
selected_features = [
    "F0semitoneFrom27.5Hz_sma3nz_stddevNorm",
    "loudness_sma3_amean",
    "HNRdBACF_sma3nz_amean",
    "jitterLocal_sma3nz_amean",
    "shimmerLocaldB_sma3nz_amean",
    "F0semitoneFrom27.5Hz_sma3nz_amean"
]

X = df[selected_features]

In [29]:
X

,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,loudness_sma3_amean,HNRdBACF_sma3nz_amean,jitterLocal_sma3nz_amean,shimmerLocaldB_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_amean
0,0.352,0.330,-1.829,0.060,1.474,21.209999
1,0.381,0.303,-1.343,0.053,1.358,20.958000
2,0.338,0.497,0.517,0.059,1.283,23.924000
3,0.281,0.621,0.590,0.040,1.239,23.309999
4,0.255,0.699,0.777,0.037,1.389,25.292999
...,...,...,...,...,...,...
72,0.397,0.422,-0.918,0.070,1.380,22.914000
73,0.295,0.587,-0.028,0.057,1.576,24.940001
74,0.174,0.610,1.345,0.040,1.466,25.489000
75,0.213,0.601,0.289,0.058,1.681,25.851999
